In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re 
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#загрузка данных
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3 )
#вывод данных
train


In [ ]:
#очистка текста от html тегов и изнаков препинания; удаление слов, не влияющих на обучение
num_reviews = train["review"].size
clean_train_reviews = []
for i in range(0, num_reviews):
    review_text = BeautifulSoup(train["review"][i]).get_text()      
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stopwordss = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stopwordss]   
    clean_train_reviews.append(" ".join(meaningful_words))

In [ ]:
#получение векторов признаков
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 20000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()

In [ ]:
#обучение при помощи рандомного леса (RandomForestClassifier) с количеством деревьев равным 150
clf = RandomForestClassifier(n_estimators=150) 
clf.fit(train_data_features, train["sentiment"])

In [ ]:
#формирование результата
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in range(0,num_reviews):
    review_text = BeautifulSoup(test["review"][i]).get_text()      
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stopwordss = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stopwordss]  
    clean_test_reviews.append(" ".join(meaningful_words))

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = clf.predict(test_data_features)

output = pd.DataFrame(data={"id":test["id"], "sentiment":result})

output.to_csv("submission.csv", index=False, quoting=3)